## 上手く動かない…

In [1]:
import datetime

import numpy as np
import pandas as pd
from scipy import optimize

import plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.graph_objs.scatter import Line
from plotly.graph_objs import Scatter

In [216]:
data = np.loadtxt('ex2data2.txt', delimiter=',')
X = data[:, [0, 1]]
y = data[:, [2]]
m = X.shape[0]
n = X.shape[1]

X = np.c_[np.ones(m), X]

X.shape, y.shape, m, n

((118, 3), (118, 1), 118, 2)

In [217]:
neg = data[data[:, 2] == 0]
pos = data[data[:, 2] == 1]

fig = go.Figure()
fig.add_trace(Scatter(x=neg[:, 0], y=neg[:, 1], mode='markers'))
fig.add_trace(Scatter(x=pos[:, 0], y=pos[:, 1], mode='markers'))
fig.show()

In [218]:
def map_feature(x1, x2):
    degree = 6
    out = np.ones(x1.shape[0])
    for i in range(1, degree + 1):
        for j in range(i + 1):
            col = (x1 ** (i-j)) * (x2 ** j)
            out = np.c_[out, col]
    return out

XX = map_feature(X[:, 0], X[:, 1])
XX

array([[ 1.00000000e+00,  1.00000000e+00,  5.12670000e-02, ...,
         6.90798869e-06,  3.54151856e-07,  1.81563032e-08],
       [ 1.00000000e+00,  1.00000000e+00, -9.27420000e-02, ...,
         7.39785525e-05, -6.86091891e-06,  6.36295342e-07],
       [ 1.00000000e+00,  1.00000000e+00, -2.13710000e-01, ...,
         2.08592830e-03, -4.45783738e-04,  9.52684426e-05],
       ...,
       [ 1.00000000e+00,  1.00000000e+00, -4.84450000e-01, ...,
         5.50802422e-02, -2.66836233e-02,  1.29268813e-02],
       [ 1.00000000e+00,  1.00000000e+00, -6.33640000e-03, ...,
         1.61201969e-09, -1.02144015e-11,  6.47225339e-14],
       [ 1.00000000e+00,  1.00000000e+00,  6.32650000e-01, ...,
         1.60196879e-01,  1.01348555e-01,  6.41181634e-02]])

In [219]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def z(th, x):
    return x @ th

def h(th, x):
    return sigmoid(z(th, x))

def J(th, x, y, lamb):
    h_th = h(th, x)
    m = x.shape[0]
    j = 1/m * ( -y * np.log(h_th) - (1-y) * np.log(1 - h_th) ).sum()
    c = lamb / (2 * m) * (th[1:] ** 2).sum()
    return j + c

def grad(th, x, y, lamb):
    g = 1/m * x.T @ (h(th, x) - y)
    c = lamb/m * th
    c[0] = 0
    return g + c.reshape(-1, 1)
    
# def costFunctionReg(th, x, y, lamb):
#     j = J(th, x, y, lamb)
#     g = grad(th, x, y, lamb)
#     return j, g

# 正規化ありのコスト関数
def costFunctionReg(theta, X, y, lambda_):
    h_theta = 1 / (1 + np.exp(np.dot(-X, theta)))
    J = np.sum(-y * np.log(h_theta) - (1 - y) * np.log(1 - h_theta)) / len(y)
    J += lambda_ / 2 / len(y) * np.sum(theta[1:] ** 2) #θ0を正則化しない

    grad = np.zeros(len(theta))
    #θ0を正則化しない
    grad[0] = np.sum((h_theta - y) * X[:, 0]) / len(y)
    for j in range(1, len(theta)):
        grad[j] = np.sum((h_theta - y) * X[:, j]) / len(y) + lambda_ / len(y) * theta[j]
    return J, grad


initial_theta = np.zeros((XX.shape[1], 1))
lamb = 1

costFunctionReg(initial_theta, XX, y, lamb)
# initial_theta.shape, X.shape, initial_theta, lamb

(0.6931471805599454,
 array([1.        , 1.        , 0.05477891, 1.        , 0.05477891,
        0.24757534, 1.        , 0.05477891, 0.24757534, 0.05983333,
        1.        , 0.05477891, 0.24757534, 0.05983333, 0.12253843,
        1.        , 0.05477891, 0.24757534, 0.05983333, 0.12253843,
        0.05196507, 1.        , 0.05477891, 0.24757534, 0.05983333,
        0.12253843, 0.05196507, 0.07837118]))

In [220]:
test_theta = np.ones((XX.shape[1], 1))
cost, g = costFunctionReg(test_theta, XX, y, 10)
cost, g

(6.511236260180357,
 array([59.65312387, 59.73786963,  3.35247878, 59.73786963,  3.35247878,
        14.8533879 , 59.73786963,  3.35247878, 14.8533879 ,  3.65399075,
        59.73786963,  3.35247878, 14.8533879 ,  3.65399075,  7.39454585,
        59.73786963,  3.35247878, 14.8533879 ,  3.65399075,  7.39454585,
         3.18462465, 59.73786963,  3.35247878, 14.8533879 ,  3.65399075,
         7.39454585,  3.18462465,  4.75983163]))

In [221]:
from scipy.optimize import fmin_bfgs

cost_f = lambda th, x, y, lamb: costFunctionReg(th, x, y, lamb)[0]

lamb = 1
result = fmin_bfgs(cost_f, initial_theta, args=(XX, y, lamb, ), full_output=True, disp=True)

theta, cost = result[0], result[1]
theta, cost

Optimization terminated successfully.
         Current function value: 81.774417
         Iterations: 24
         Function evaluations: 870
         Gradient evaluations: 29


(array([-3.29763259e-02, -1.53906054e-04,  1.85468422e-06, -1.53906054e-04,
         1.85468422e-06, -1.72190849e-06, -1.53906054e-04,  1.85468422e-06,
        -1.72190849e-06, -9.40773582e-06, -1.53906054e-04,  1.85468422e-06,
        -1.72190849e-06, -9.40773582e-06, -9.60055881e-06, -1.53906054e-04,
         1.85468422e-06, -1.72190849e-06, -9.40773582e-06, -2.02679149e-06,
         9.30922990e-06, -1.53906054e-04,  1.85468422e-06, -1.72190849e-06,
        -9.40773582e-06, -2.02679149e-06,  9.30922990e-06,  3.70251513e-05]),
 81.77441734267147)

In [222]:
def plotDecisionBoundary(theta, X, y):
    pos_x1 = X[:, [1]][y == 1]
    pos_x2 = X[:, [2]][y == 1]
    neg_x1 = X[:, [1]][y == 0]
    neg_x2 = X[:, [2]][y == 0]

    fig = go.Figure()
    fig.add_trace(Scatter(x=neg_x1, y=neg_x2, mode='markers'))
    fig.add_trace(Scatter(x=pos_x1, y=pos_x2, mode='markers'))
    fig.show()
    
# plot_x1 = np.linspace(X[:, 1].min(0), X[:, 1].max(0), 100)
u = np.linspace(-1, 1.5, 50)
v = np.linspace(-1, 1.5, 50)
z = np.zeros((len(v), len(u)))
for i in range(len(u)):
    for j in range(len(v)):
            z[j, i] = map_feature(np.array([u[i]]), np.array([v[j]])) @ theta

fig = go.Figure()
fig.add_trace(go.Contour(z=z))
fig.show()

    
# X[:, 1].reshape(-1, 1)[y==1]
# pos_x1 = X[:, 1][y == 1]

    

plotDecisionBoundary(theta, X, y)